In [1]:
from typing import Dict
from sklearn.model_selection import train_test_split

from dataclasses import dataclass, field
from typing import cast, Optional

import torch
from datasets import load_dataset
from torch.utils.data import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    HfArgumentParser,
    PreTrainedTokenizer
)
from transformers.trainer_pt_utils import LabelSmoother
from transformers.integrations import deepspeed

from peft import LoraConfig, get_peft_model

import os

os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "9994"  # modify if RuntimeError: Address already in use
os.environ["RANK"] = "0"
os.environ["LOCAL_RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"

[2023-10-14 20:10:22,476] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
IGNORE_TOKEN_ID = LabelSmoother.ignore_index

In [3]:
from huggingface_hub import notebook_login
 
notebook_login()

In [3]:
import pandas as pd

csv_files = ["mental_health_chatbot_dataset.csv", "psychology-dataset.csv", "who_r_u.csv"]
df = pd.DataFrame()

for p in csv_files:
    df1 = pd.read_csv(f"../../data/processed/{p}")[["human", "zen"]]
    df1 = df1.rename(columns={"human": "USER", "zen": "ASSISTANT"})
    df1 = df1.drop_duplicates(subset=["USER", "ASSISTANT"], keep="first", ignore_index=True)

    df1["USER"] = df1.USER.str.replace('\s+', ' ', regex=True)
    df1["USER"] = df1.USER.str.replace(r'\.([a-zA-Z0-9])', r'. \1', regex=True)
    df1["ASSISTANT"] = df1.ASSISTANT.str.replace('\s+', ' ', regex=True)
    df1["ASSISTANT"] = df1.ASSISTANT.str.replace(r'\.([a-zA-Z0-9])', r'. \1', regex=True)
    
    df = pd.concat([df, df1])

df = df.sample(frac=1, random_state=42)
train, test = train_test_split(df, test_size=1000, random_state=42)
df.shape, train.shape, test.shape

((10486, 2), (9486, 2), (1000, 2))

In [4]:
def format_example(df):
    instruction = (
        "Your name is ZenAI and you're a licensed therapist. Please have a conversation with your patient and provide them with a helpful response to their concerns."
    )
    p_header = "USER:"
    z_header = "ASSISTANT:"

    p = df["USER"]
    z = df["ASSISTANT"]

    df[
        "text"
    ] = f"{instruction} {p_header} {p} {z_header} {z}</s>"

    return df

In [5]:
def preprocess(conversations, tokenizer):

    # Tokenize conversations
    input_ids = tokenizer(
        conversations,
        return_tensors="pt",
        padding="max_length",
        max_length=tokenizer.model_max_length,
        truncation=True,
    ).input_ids
    targets = input_ids.clone()

    # Mask targets. Only compute loss on the assistant outputs.
    sep = " ASSISTANT: "
    for conversation, target in zip(conversations, targets):
        total_len = int(target.ne(tokenizer.pad_token_id).sum())

        turns = conversation.split("</s>")
        cur_len = 1
        target[:cur_len] = IGNORE_TOKEN_ID
        for i, turn in enumerate(turns):
            if turn == "":
                break
            turn_len = len(tokenizer(turn).input_ids)

            parts = turn.split(sep)
            if len(parts) != 2:
                break
            parts[0] += sep
            # "-2" is hardcoded for the Llama tokenizer to make the offset correct.
            instruction_len = len(tokenizer(parts[0]).input_ids) - 2

            if i != 0 and not tokenizer.legacy:
                # The legacy and non-legacy modes handle special tokens differently
                instruction_len -= 1

            # Ignore the user instructions
            target[cur_len : cur_len + instruction_len] = IGNORE_TOKEN_ID
            cur_len += turn_len

            if i != 0 and not tokenizer.legacy:
                # The legacy and non-legacy modes handle special tokens differently
                cur_len -= 1

        target[cur_len:] = IGNORE_TOKEN_ID

        if False:  # Inspect and check the correctness of masking
            z = target.clone()
            z = torch.where(z == IGNORE_TOKEN_ID, tokenizer.unk_token_id, z)
            rank0_print(tokenizer.decode(z))
            exit()

        if cur_len < tokenizer.model_max_length:
            if cur_len != total_len:
                target[:] = IGNORE_TOKEN_ID
                rank0_print(
                    f"WARNING: tokenization mismatch: {cur_len} vs. {total_len}."
                    f" #turn = {len(turns) - 1}. (ignored)"
                )

    return dict(
        input_ids=input_ids,
        labels=targets,
        attention_mask=input_ids.ne(tokenizer.pad_token_id),
    )

In [6]:
model_id = "lmsys/vicuna-13b-v1.5"

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    model_max_length=4096,
    padding_side="right",
    use_fast=False,
)
tokenizer.pad_token = tokenizer.unk_token

In [7]:
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, df, tokenizer: PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()

        conversations = df.sample(n=100).apply(format_example, axis=1).text.tolist()
        data_dict = preprocess(conversations, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        self.attention_mask = data_dict["attention_mask"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(
            input_ids=self.input_ids[i],
            labels=self.labels[i],
            attention_mask=self.attention_mask[i],
        )

In [8]:
train_dataset = SupervisedDataset(train, tokenizer)
eval_dataset = SupervisedDataset(test, tokenizer)
data_module = dict(train_dataset=train_dataset, eval_dataset=eval_dataset)

In [9]:
# train_args = TrainingArguments(
#     output_dir="models/zenai_sample",
#     max_steps=10000,
#     optim="adamw_torch",
#     per_device_train_batch_size=1,
#     remove_unused_columns=False,
#     learning_rate=1e-5,
#     gradient_checkpointing=True,
#     gradient_accumulation_steps=1,
#     fp16=True,
#     lr_scheduler_type="constant_with_warmup",
#     warmup_steps=100,
#     save_steps=1000,
#     save_strategy="steps",
#     evaluation_strategy="steps",
#     eval_steps=1000,
#     save_total_limit=3,
#     logging_steps=100,
#     report_to="tensorboard",
#     hub_model_id="kmnis/ZenAI-v2",
#     hub_strategy="checkpoint",
#     hub_private_repo=True,
#     load_best_model_at_end=True,
#     push_to_hub=True,
#     deepspeed="ds_config_zero3.json"
# )

train_args = TrainingArguments(
    output_dir="/home/jupyter/therapy-bot/models/zenai_sample",
    max_steps=200,
    optim="adamw_torch",
    per_device_train_batch_size=1,
    remove_unused_columns=False,
    learning_rate=1e-5,
    gradient_checkpointing=True,
    gradient_accumulation_steps=1,
    fp16=True,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=5,
    save_steps=50,
    save_strategy="steps",
    evaluation_strategy="steps",
    eval_steps=50,
    save_total_limit=3,
    logging_steps=10,
    report_to="tensorboard",
    hub_model_id="kmnis/ZenAI-v2",
    hub_strategy="checkpoint",
    hub_private_repo=True,
    load_best_model_at_end=True,
    push_to_hub=True,
    deepspeed="ds_config_zero3.json"
)

lora_r = 8
lora_alpha = 16
lora_dropout = 0.1
target_modules=["q_proj", "k_proj", "v_proj", "out_proj", "fc_in", "fc_out", "wte"]

[2023-10-14 20:11:04,558] [INFO] [comm.py:637:init_distributed] cdb=None
[2023-10-14 20:11:04,559] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl


In [10]:
# preparing lora configuration
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CASUAL_LM",
    target_modules=target_modules,
)

# loading the base model
model = AutoModelForCausalLM.from_pretrained(
    model_id, use_cache=not train_args.gradient_checkpointing
)
if train_args.gradient_checkpointing:
    model.gradient_checkpointing_enable()

# getting peft model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# creating trainer
trainer = Trainer(
    model=model, tokenizer=tokenizer, args=train_args, **data_module
)
model.config.use_cache = False

# trainer.accelerator.print(f"{trainer.model}")
trainer.model.print_trainable_parameters()

[2023-10-14 20:11:44,545] [INFO] [partition_parameters.py:347:__exit__] finished initializing model - num_params = 363, num_elems = 13.02B


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


trainable params: 9,830,400 || all params: 13,025,694,720 || trainable%: 0.07546929519932738
trainable params: 9,830,400 || all params: 13,025,694,720 || trainable%: 0.07546929519932738


/opt/conda/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [1]:
%load_ext tensorboard
%tensorboard --logdir /home/jupyter/therapy-bot/models/zenai_sample/runs/

In [12]:
# start training
trainer.train()

Using /home/jupyter/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/jupyter/.cache/torch_extensions/py310_cu118/cpu_adam/build.ninja...
Building extension module cpu_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module cpu_adam...


ninja: no work to do.
Time to load cpu_adam op: 2.4352633953094482 seconds
Parameter Offload: Total persistent parameters: 10245120 in 321 params


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


[2023-10-14 20:14:13,938] [WARNING] [stage3.py:1936:step] 31 pytorch allocator cache flushes since last step. this happens when there is high memory pressure and is detrimental to performance. if this is happening frequently consider adjusting settings to reduce memory consumption. If you are unable to make the cache flushes go away consider adding get_accelerator().empty_cache() calls in your training loop to ensure that all ranks flush their caches at the same time


/opt/conda/lib/python3.10/site-packages/deepspeed/runtime/zero/stage3.py:1274: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  total_norm_cuda = get_accelerator().FloatTensor([float(total_norm)])


Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.50 GiB. GPU 0 has a total capacty of 14.58 GiB of which 95.38 MiB is free. Including non-PyTorch memory, this process has 14.40 GiB memory in use. Of the allocated memory 10.87 GiB is allocated by PyTorch, and 2.56 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# save model on main process
trainer.accelerator.wait_for_everyone()
state_dict = trainer.accelerator.get_state_dict(trainer.deepspeed)
unwrapped_model = trainer.accelerator.unwrap_model(trainer.deepspeed)
if trainer.accelerator.is_main_process:
    unwrapped_model.save_pretrained(train_args.output_dir, state_dict=state_dict)
trainer.accelerator.wait_for_everyone()

# save everything else on main process
if trainer.args.process_index == 0:
    trainer.model.save_pretrained(train_args.output_dir, safe_serialization=True)

In [ ]:
trainer.push_to_hub()